In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Validation Set Approach

In [3]:
### We want to estimate the test error rates that results from fitting various linear models on the auto data set
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split(Auto , test_size=196, random_state=0)

In [4]:
# we fit the linear regression using just the training dataset
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [6]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
# MSE of the model
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [7]:
### We can also estimate the validation error for higher degree polynomial regressions by creating the following function
def evalMSE(terms ,response ,train , test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [9]:
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [10]:
Auto_train , Auto_valid = train_test_split(Auto ,test_size=196, random_state=3)
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],'mpg',Auto_train , Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])